In [ ]:
#!pip install -r requirements.txt
!pip install streamlit
!pip install langchain
!pip install PyPDF2
!pip install faiss-cpu
!pip install langchain_google_genai
!pip install python-dotenv
!pip install -q streamlit
!pip install pyperclip

In [ ]:
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

In [ ]:
load_dotenv()
os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [ ]:
import os

def new_get_pdf_text(pdf_dir):
  text = ""
  for filename in os.listdir(pdf_dir):
    if filename.endswith(".pdf"):  # Check for PDF extension
      filepath = os.path.join(pdf_dir, filename)
      with open(filepath, 'rb') as f:
        pdf_reader = PdfReader(f)
        for page in pdf_reader.pages:
          text += page.extract_text()
  return text

In [ ]:
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    chunks = text_splitter.split_text(text)
    return chunks

In [ ]:
def get_vector_store(text_chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")

In [ ]:
'''
prompt_template = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """
'''

def get_conversational_chain():

    prompt_template = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details,\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """

    model = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.3)

    prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

    return chain

In [ ]:
def user_input(user_question):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

    new_db = FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)
    docs = new_db.similarity_search(user_question)

    chain = get_conversational_chain()


    response = chain(
        {"input_documents":docs, "question": user_question}
        , return_only_outputs=True)

    print(response['output_text'])

In [ ]:
pdf_dir = "/content/"
raw_text = new_get_pdf_text(pdf_dir)
text_chunks = get_text_chunks(raw_text)
get_vector_store(text_chunks)

In [ ]:
user_question = "tell me about the second unit of software engineering subject"
if user_question:
  user_input(user_question)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


**Unit - II: Advanced Process Models & Tools**

* Agile software development: Agile methods, Plan-driven and agile development, Extreme programming Practices, Testing in XP, Pair programming.
* Introduction to agile tools: JIRA, Kanban, Case Studies: An information system (mental health-care system), wilderness weather system


In [ ]:
%%writefile app.py
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

from google.colab import userdata
api = userdata.get('google_api')

load_dotenv()
os.getenv("GOOGLE_API_KEY")
genai.configure(api_key= api)


import os

def get_pdf_text(pdf_dir):
    text = ""
    for filename in os.listdir(pdf_dir):
        if filename.endswith(".pdf"):  # Check for PDF extension
            filepath = os.path.join(pdf_dir, filename)
            with open(filepath, 'rb') as f:
                pdf_reader = PdfReader(f)
                for page in pdf_reader.pages:
                    text += page.extract_text()
    return text


def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    chunks = text_splitter.split_text(text)
    return chunks


def get_vector_store(text_chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")


def get_conversational_chain():
    prompt_template = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """

    model = ChatGoogleGenerativeAI(model="gemini-pro",
                                   temperature=0.3)

    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

    return chain


def user_input(user_question):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

    new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
    docs = new_db.similarity_search(user_question)

    chain = get_conversational_chain()

    response = chain(
        {"input_documents": docs, "question": user_question}
        , return_only_outputs=True)

    print(response)
    st.write(response["output_text"])


def main():
    st.set_page_config("Student Chatbot")
    st.title("Student Chatbot")


    pdf_dir = "/content/"
    raw_text = get_pdf_text(pdf_dir)
    text_chunks = get_text_chunks(raw_text)
    get_vector_store(text_chunks)

    st.write("Welcome to the Student Chatbot! Ask any questions you have about your syllabus..")

    user_question = st.text_input("Ask your question here:")

    if user_question:
        user_input(user_question)

    st.success("Done")


if __name__ == "__main__":
    main()

Writing app.py


In [ ]:
%%writefile app.py
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import pyperclip

load_dotenv()
os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))


import os

def get_pdf_text(pdf_dir):
    text = ""
    for filename in os.listdir(pdf_dir):
        if filename.endswith(".pdf"):  # Check for PDF extension
            filepath = os.path.join(pdf_dir, filename)
            with open(filepath, 'rb') as f:
                pdf_reader = PdfReader(f)
                for page in pdf_reader.pages:
                    text += page.extract_text()
    return text


def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    chunks = text_splitter.split_text(text)
    return chunks


def get_vector_store(text_chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")


def get_conversational_chain():
    prompt_template = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """

    model = ChatGoogleGenerativeAI(model="gemini-pro",
                                   temperature=0.3)

    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

    return chain


def user_input(user_question):
    # Check if the input is a string
    if not isinstance(user_question, str):
        st.error("Error: Input must be a string.")
        return

    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

    new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
    docs = new_db.similarity_search(user_question)

    chain = get_conversational_chain()

    response = chain(
        {"input_documents": docs, "question": user_question}
        , return_only_outputs=True)

    # Display the output text
    st.write(response["output_text"])

    # Add a button to copy the output text to the clipboard
    if st.button("Copy Output"):
        pyperclip.copy(response["output_text"])
        st.success("Output text copied to clipboard")


def main():
    st.set_page_config("Student Chatbot")
    st.title("Student Chatbot")

    pdf_dir = "/content/"
    raw_text = get_pdf_text(pdf_dir)
    text_chunks = get_text_chunks(raw_text)
    get_vector_store(text_chunks)

    st.write("Welcome to the Student Chatbot! Ask any questions you have about your syllabus..")

    st.markdown("### Example question: tell me about the second unit of software engineering subject")

    # Add placeholder and label for the input field
    user_question = st.text_input("Ask your question here:", placeholder="Question")

    if user_question:
        user_input(user_question)

    st.success("Done")


if __name__ == "__main__":
    main()


Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.127.86.134:8501

npx: installed 22 in 2.999s
your url is: https://rotten-camels-cheer.loca.lt
/usr/local/lib/python3.10/dist-packages/langchain/vectorstores/__init__.py:35: LangChainDeprecationWarning: Importing vector stores from langchain is deprecated. Importing from langchain will no longer be supported as of langchain==0.2.0. Please import from langchain-community instead:

`from langchain_community.vectorstores import FAISS`.

To install langchain-community run `pip install -U langchain-community`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain/vectorstores/__init__.py:35: LangChainDeprecationWarning: I